# 🚀  Comparison of CNN Architectures on Different Datasets


This notebook compares CNN architectures (**AlexNet, VGG16, GoogLeNet, ResNet18, Xception, SENet**) on three datasets (**MNIST, FMNIST, CIFAR-10**) using:

- ✅ Subset-based training (30%)
- ✅ `torch.compile` for optimization
- ✅ AMP (Automatic Mixed Precision)
- ✅ Summary tables for accuracy and F1-score

In [ ]:
!pip install timm

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import timm
from collections import defaultdict
from torch.cuda.amp import autocast, GradScaler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

transform_common = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_cifar = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

datasets_dict = {
    "MNIST": {
        "train": datasets.MNIST(root="./data", train=True, download=True, transform=transform_common),
        "test": datasets.MNIST(root="./data", train=False, download=True, transform=transform_common)
    },
    "FMNIST": {
        "train": datasets.FashionMNIST(root="./data", train=True, download=True, transform=transform_common),
        "test": datasets.FashionMNIST(root="./data", train=False, download=True, transform=transform_common)
    },
    "CIFAR10": {
        "train": datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_cifar),
        "test": datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_cifar)
    }
}


In [ ]:

def get_subset_loader(dataset, percent=0.3, batch_size=32):
    subset_len = int(len(dataset) * percent)
    subset = Subset(dataset, list(range(subset_len)))
    return DataLoader(subset, batch_size=batch_size, shuffle=True)


In [ ]:

def get_model(name, num_classes=10):
    if name == "alexnet":
        model = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif name == "vgg16":
        model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    elif name == "googlenet":
        model = models.googlenet(weights=models.GoogLeNet_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif name == "xception":
        model = timm.create_model('xception', pretrained=True)
        model.reset_classifier(num_classes)
    elif name == "senet":
        model = timm.create_model('seresnet18', pretrained=True)
        model.reset_classifier(num_classes)
    else:
        raise ValueError("Unsupported model name")
    return torch.compile(model)


In [ ]:

def train_model(model, loader, optimizer, criterion, device, epochs=3):
    model.to(device)
    model.train()
    scaler = GradScaler()
    for epoch in range(epochs):
        total_loss = 0.0
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

def evaluate_model(model, loader, device):
    model.to(device)
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    report = classification_report(y_true, y_pred, output_dict=True)
    return report


## 🌀 Training Loop & Summary Table

In [ ]:

results = defaultdict(dict)
datasets_to_run = ["MNIST", "FMNIST", "CIFAR10"]
models_to_run = ["alexnet", "vgg16", "googlenet", "resnet18", "xception", "senet"]

for dataset_name in datasets_to_run:
    print(f"\n=== Dataset: {dataset_name} ===")
    train_loader = get_subset_loader(datasets_dict[dataset_name]["train"])
    test_loader = DataLoader(datasets_dict[dataset_name]["test"], batch_size=32)

    for model_name in models_to_run:
        print(f"\n→ Training {model_name} on {dataset_name}")
        model = get_model(model_name)
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        train_model(model, train_loader, optimizer, criterion, device, epochs=3)

        print(f"→ Evaluating {model_name} on {dataset_name}")
        report = evaluate_model(model, test_loader, device)
        results[dataset_name][model_name] = report


In [ ]:

print("\n\n===== 📊 Summary Table =====")
for dataset in results:
    print(f"\nDataset: {dataset}")
    for model in results[dataset]:
        acc = results[dataset][model]["accuracy"]
        f1 = results[dataset][model]["weighted avg"]["f1-score"]
        print(f"{model:10} | Accuracy: {acc:.4f} | F1-score: {f1:.4f}")
